In [ ]:
import spacy
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00


In [ ]:
import json
#!pip install spacy-transformers
import spacy_transformers
with open('train.json', 'r') as file:
    ronec = json.load(file)
ronec=ronec[0:len(ronec)]

In [ ]:
from spacy.training.example import Example
nlp = spacy.blank("ro")
entity_labels = [
    "O","PERSON", "GPE", "LOC", "ORG", "LANGUAGE", "NAT_REL_POL",
    "DATETIME", "PERIOD", "QUANTITY", "MONEY", "NUMERIC",
    "ORDINAL", "FACILITY", "WORK_OF_ART", "EVENT"
]
#for label in entity_labels:
 #   nlp.vocab.strings.add(label)

training_data=[]
train_examples = []
ronec_train=ronec[0:len(ronec)//2]
for data in ronec_train:
    start=0
    entities = []
    text=""
    for token, label, space in zip(data['tokens'], data['ner_tags'], data["space_after"]):
        entities.append((start, start+len(token), label))
        text+=token
        if space:
          text+=" "
          start+=1
        #print((start, start+len(token),label,token))
        start+=len(token)
    training_data.append({"text": text, "entities":entities})
print(training_data[0])

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


{'text': 'În România, ca de altfel în întreaga Europă, stand up-ul este forma de divertisment cea mai îndrăgită de tineri în acest moment, comediantii reușind să umple săli care erau destinate mai degrabă artiștilor din zona muzicii pop rock, iar Badea, Bordea și Micutzu sunt printre cei care au contribuit direct la succesul acestei forme de divertisment în țara noastră.', 'entities': [(0, 2, 'O'), (3, 10, 'GPE'), (10, 11, 'O'), (12, 14, 'O'), (15, 17, 'O'), (18, 24, 'O'), (25, 27, 'O'), (28, 36, 'O'), (37, 43, 'LOC'), (43, 44, 'O'), (45, 50, 'O'), (51, 53, 'O'), (53, 56, 'O'), (57, 61, 'O'), (62, 67, 'O'), (68, 70, 'O'), (71, 83, 'O'), (84, 87, 'O'), (88, 91, 'O'), (92, 101, 'O'), (102, 104, 'O'), (105, 111, 'PERSON'), (112, 114, 'O'), (115, 120, 'O'), (121, 127, 'O'), (127, 128, 'O'), (129, 140, 'PERSON'), (141, 148, 'O'), (149, 151, 'O'), (152, 157, 'O'), (158, 162, 'O'), (163, 167, 'O'), (168, 172, 'O'), (173, 182, 'O'), (183, 186, 'O'), (187, 194, 'O'), (195, 205, 'PERSON'), (206,

In [ ]:
from spacy.util import filter_spans
from spacy.tokens import DocBin
from tqdm import tqdm

doc_bin = DocBin()

for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label)
        if span is None:
            print(start,end,label,text[start:end],"no")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

  1%|          | 41/6165 [00:00<00:33, 184.48it/s]

156 157 QUANTITY m no
157 158 O . no
42 48 NAT_REL_POL austro no
48 49 O - no
49 56 NAT_REL_POL germană no
237 243 LOC Isonzo no
243 244 O ; no
244 249 O focul no
150 151 NUMERIC I no
151 152 O - no
34 36 ORDINAL 12 no
36 38 ORDINAL -a no
34 35 O r no
35 43 NAT_REL_POL omânilor no
43 44 O / no
44 52 NAT_REL_POL vlahilor no
192 193 O a no
193 200 O Agenția no
368 370 O și no
370 376 GPE Turcia no


  1%|          | 60/6165 [00:00<00:45, 134.23it/s]

584 587 O top no
587 588 O ) no
588 589 O . no
589 594 O Piesa no
70 73 NUMERIC 110 no
73 74 O / no
74 78 DATETIME 2005 no
305 308 NUMERIC 515 no
308 309 O / no
309 319 DATETIME 14.06.2006 no
44 52 PERSON muzicant no
52 53 O ” no
53 54 O ( no
54 58 O F.P. no
58 59 O ) no
59 60 O . no
60 71 O Obiceiurile no
30 31 O ) no
31 32 O : no
164 165 O ) no
165 166 O : no
187 194 O știință no
194 197 O Iar no


  1%|          | 75/6165 [00:00<00:55, 109.79it/s]

...

 Orient no
314 315 O » no
315 316 O ” no
316 317 O [ no
317 319 NUMERIC 30 no
319 320 O ] no
320 321 O . no
321 323 O În no
486 494 O militare no
494 495 O ” no
495 496 O [ no
496 498 NUMERIC 31 no
64 73 NAT_REL_POL Elvețiano no
73 74 O - no
74 79 NAT_REL_POL Român no
63 67 DATETIME 2000 no
67 68 O , no
68 72 O trad no
30 35 WORK_OF_ART ACASA no
35 36 WORK_OF_ART " no
36 38 O Se no
42 45 O tel no
45 46 O . no
54 58 DATETIME 2002 no
58 59 O . no
59 65 O Pentru no
39 41 LOC Bd no
41 42 LOC . no
122 125 NUMERIC 153 no
125 126 O / no
126 130 DATETIME 2017 no
120 122 LOC Bd no
122 123 LOC . no
319 323 DATETIME 2009 no
323 324 O . no
324 328 O FOTO no
439 450 ORG Lingvistică no
450 451 ORG „ no
451 463 ORG Argumentarea no
168 178 NAT_REL_POL israeliano no
178 179 O - no
179 190 NAT_REL_POL palestinian no
70 71 PERSON B no
71 72 O . no
112 121 PERIOD 2016-2018 no
121 122 O . no
122 131 O Seminarul no
185 194 ORG Gimnaziul no
194 195 ORG , no
195 196 ORG , no
196 198 ORG A. no
221 225 O tema n

 99%|█████████▊| 6085/6165 [00:17<00:00, 406.79it/s]

 108 LOC . no
228 229 O „ no
229 230 O O no
276 286 NAT_REL_POL evanghelic no
286 287 O ” no
350 351 O a no
351 354 O -și no
383 384 O a no
384 387 O -și no
64 70 GPE Italia no
70 71 O - no
71 77 O acesta no
114 123 NAT_REL_POL latinilor no
123 124 O - no
124 130 DATETIME epocii no
167 174 NAT_REL_POL italian no
174 179 PERSON Dante no
180 189 PERSON Alighieri no
189 190 O a no
147 148 O „ no
148 155 O Începem no
164 165 QUANTITY m no
165 166 O . no
476 477 O . no
477 483 O Numele no
103 106 LOC str no
106 107 LOC . no
106 107 O “ no
107 109 O În no
89 90 NUMERIC 5 no
90 91 O ) no
91 92 O . no
92 100 PERSON Belșațar no
151 155 O post no
155 156 O - no
156 161 PERSON Fidel no
99 108 PERIOD 1966-1967 no
108 109 O , no
109 110 NUMERIC 3 no
174 183 PERIOD 1964-1965 no
183 184 O , no
184 185 NUMERIC 3 no
203 212 PERIOD 1967-1968 no
212 213 O , no
213 214 NUMERIC 2 no
153 157 DATETIME 2018 no
157 158 O . no
158 160 O Ne no
43 44 O S no
44 46 NUMERIC 10 no
103 107 O țară no
107 108 O ; no
108

100%|██████████| 6165/6165 [00:17<00:00, 352.27it/s]


 O foi no
96 97 O " no
97 98 O ( no
98 104 O pănuși no
74 76 PERSON dr no
76 77 PERSON . no
175 182 O medical no
182 183 O - no
183 190 NAT_REL_POL creștin no
216 224 GPE Michigan no
224 225 O ” no
226 227 O „ no
227 233 O Echipa no
222 229 O comandă no
229 232 O -că no
124 130 NAT_REL_POL români no
130 136 O Obiect no
11 21 PERSON publicului no
21 29 O accident no
288 290 QUANTITY mp no
290 304 O Amendaapararea no
1305 1312 PERSON monahal no
1312 1336 O clopotnițelorConstrucții no
1568 1570 QUANTITY mp no
1570 1572 O cu no
1606 1608 QUANTITY mp no
1608 1610 O cu no
1654 1656 QUANTITY mp no
1656 1658 O cu no
1702 1704 QUANTITY mp no
1704 1706 O cu no
1750 1752 QUANTITY mp no
1752 1754 O cu no
2050 2052 QUANTITY mp no
2052 2056 O dacă no
2206 2208 QUANTITY mp no
2208 2210 O de no
0 2 O De no
2 4 O -a no
95 105 PERSON profesorii no
105 106 O / no
106 119 PERSON coordonatorii no


In [ ]:
!python -m spacy init fill-config base-config.cfg config.cfg


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy  --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        1064.35    171.10    0.50    0.83    0.36    0.01
  1     200   

In [ ]:
test_data=ronec[len(ronec)-50:len(ronec)]
predicted_entities=[]

spacy.prefer_gpu()
nlp_ner = spacy.load("content/model-best")

for data in test_data:
  text=""
  for cuv, space in zip(data["tokens"],data["space_after"]):
    text+=cuv
    print(cuv)
    #if space:
    text+=" "
  doc = nlp_ner(text)
  predicted_entities.extend( [(ent.text,ent.ent_type_) for ent in doc])

  #print(text)


print(predicted_entities)

Intră
în
vigoare
Planul
politicii
naționale
a
Republicii
Moldova
,
prin
care
românii
sunt
declarați
minoritari
în
acest
stat
Antrenamentele
se
țin
zilnic
,
de
luni
până
vineri
,
începând
cu
ora
20
la
Grădinița
nr.
26
de
pe
strada
Teiului
nr.
3
(
vizavi
de
arena
"
Viola
"
)
.
Lunar
,
beneficiarii
vor
primi
600
de
euro
și
le
va
fi
asigurată
cazarea
,
transportul
către
și
din
țara
de
rezidență
.
Din
câte
se
pare
,
primul
pictor
român
care
a
descoperit
Balcicul
a
fost
Iosif
Iser
în
1913
,
când
participa
ca
soldat
la
războiul
balcanic
.
De
asemenea
în
Centuria
a
Șasea
,
aparută
în
anul
1559
la
Salonic
,
a
formulat
un
jurământ
al
medicului
,
în
tradiția
medicilor
evrei
ai
epocii
.
Când
avea
numai
doi
ani
,
a
fost
promisă
Delfinului
Francisc
al
Franței
,
fiul
regelui
Francisc
I
al
Franței
însă
numai
după
trei
ani
contractul
a
fost
respins
.
În
capitolul
119
(
The
Candles
)
din
Moby-Dick
de
Herman
Melville
,
Ishmael
descrie
prevestitor
apariția
Focului
Sfanțului
Elmo
(
"
The
corpusants
!
"
)
p

In [ ]:
from sklearn.metrics import classification_report

target=[]
cuvs=[]

for data in test_data:
  target.extend(data['ner_tags'])
  cuvs.extend(data["tokens"])

print(len(target),len(cuvs))



test=[ x for x in predicted_entities if x[0]!="'"]
test=[x for x in zip(cuvs,predicted_entities) if x[0]!=x[1]]

print(test)


report = classification_report(target, predicted_entities)
print(report)

1976 1976
[('Intră', ('Intră', 'O')), ('în', ('în', 'O')), ('vigoare', ('vigoare', 'O')), ('Planul', ('Planul', 'O')), ('politicii', ('politicii', 'O')), ('naționale', ('naționale', 'O')), ('a', ('a', 'O')), ('Republicii', ('Republicii', 'GPE')), ('Moldova', ('Moldova', 'GPE')), (',', (',', 'O')), ('prin', ('prin', 'O')), ('care', ('care', 'O')), ('românii', ('românii', 'NAT_REL_POL')), ('sunt', ('sunt', 'O')), ('declarați', ('declarați', 'O')), ('minoritari', ('minoritari', 'O')), ('în', ('în', 'O')), ('acest', ('acest', 'O')), ('stat', ('stat', 'O')), ('Antrenamentele', ('Antrenamentele', 'O')), ('se', ('se', 'O')), ('țin', ('țin', 'O')), ('zilnic', ('zilnic', 'DATETIME')), (',', (',', 'O')), ('de', ('de', 'PERIOD')), ('luni', ('luni', 'PERIOD')), ('până', ('până', 'PERIOD')), ('vineri', ('vineri', 'PERIOD')), (',', (',', 'O')), ('începând', ('începând', 'O')), ('cu', ('cu', 'O')), ('ora', ('ora', 'DATETIME')), ('20', ('20', 'DATETIME')), ('la', ('la', 'O')), ('Grădinița', ('Grădiniț

ValueError: Found input variables with inconsistent numbers of samples: [1976, 1981]

In [ ]:
!cp "/content/drive/MyDrive/model-best.zip" "/content/model-best.zip"

In [ ]:
!unzip "/content/model-best.zip"

Archive:  /content/model-best.zip
   creating: content/model-best/
   creating: content/model-best/transformer/
 extracting: content/model-best/transformer/cfg  
  inflating: content/model-best/transformer/model  
   creating: content/model-best/ner/
  inflating: content/model-best/ner/moves  
  inflating: content/model-best/ner/cfg  
  inflating: content/model-best/ner/model  
  inflating: content/model-best/tokenizer  
   creating: content/model-best/vocab/
 extracting: content/model-best/vocab/key2row  
 extracting: content/model-best/vocab/lookups.bin  
 extracting: content/model-best/vocab/vectors.cfg  
  inflating: content/model-best/vocab/vectors  
  inflating: content/model-best/vocab/strings.json  
  inflating: content/model-best/config.cfg  
  inflating: content/model-best/meta.json  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
